In [ ]:
from random import gauss
from math import log, e, sqrt, pi
from scipy import stats

# 一般期权的初始参数
s0 = 60                 # 标的资产当前价格
T = 126/252              # 期权到期时间
r = 0.0500              # 无风险收益率
q = 0                   # 股息率
sigma = 0.20            # 标的资产年化波动率
strike = 65             # 期权的行权价格
option_type = 'call'    # 期权类型，认沽：put， 认购：call

# 模拟过程的参数
n_steps = 200           # 每次模拟的步数
N_simulations = 1000     # 模拟价格路径的次数
barrie_price = 68       # 障碍价格

def get_vanilla_price(S, X, r, q, sigma, T, opt_type='call'):
    """
    根据BSM模型计算普通欧式期权价格
    param S: 标的资产当前价格
    param X: 期权行权价格
    param r: 无风险收益率
    param q: 基础资产分红收益率
    param sigma：标的资产年化波动率
    param T: 以年计算的期权到期时间
    param opt_type: 'call'或'put'
    return 返回期权权益金
    """
    d1 = (log(S/X) + (r - q + sigma**2/2) * T) / (sigma * sqrt(T))
    d2 = d1 - sigma * sqrt(T)
    N1, N2 = stats.norm.cdf([d1, d2])
    call = S * e ** (-q * T) * N1 - X * e ** (-r * T) * N2      # 计算认购期权价格
             
    if opt_type.lower()[0] == 'c':
        return call
    else: 
        put = call + X * e ** (-r * T) - S * e ** (-q * T)  # 利用期权平价模型计算put价格
        return put

# 通过蒙特卡洛模拟实现定价的思路是模拟N条价格路径，在每条价格路径上计算期权到期时的价值，对所有价格路径上的期权价值取平均值后折现到当前
# 实现过程中，障碍期权相对于普通期权的区别在于，要根据不同价格路径上的情况判断障碍期权是否会生效。
def get_barrier_prices(s0, X, r, q, sigma, T, option_type, barrier_price, barrier_direction):
    """
    定义函数计算障碍期权的价值
    param s0: 标的资产当前价格
    param X: 期权行权价格
    param r: 无风险收益率
    param q: 基础资产分红收益率
    param sigma：标的资产年化波动率
    param T: 以年计算的期权到期时间
    param option_type: 'call'或'put'
    param barrier_price: 障碍价格
    param barrier_direction: 障碍价格运动方向
    return 返回期期权权益金
    """
    N_simulations = 1000
    n_steps = 200
    dt = T / n_steps
    
    vanilla = get_vanilla_price(s0, X, r, q, sigma, T, option_type) #障碍期权如果生效则相当于普通欧式期权
    
    in_sum = 0
    out_sum = 0
    
    # 模拟 N_simulations 条价格路径：
    for i in range(N_simulations):
        s = s0            # 每次模拟标的资产价格都从当前价格开始演化                                     
        active = False    # 初始状态障碍期权不生效，即 knock-in 的情况

        # 模拟标的资产价格的几何布朗运动，确定不同价格路径下障碍条件是否会触及
        for j in range(n_steps):
            z = gauss(0,1)
            # 资产价格的几何布朗运动
            s *= e ** ((r - 0.5*(sigma**2))*dt + sigma*sqrt(dt)*z)
            # 针对不同的敲入/敲出条件采用不同的逻辑
            if barrier_direction.lower()[0] == 'd':        # 对应下跌敲入或敲出的情况 
                if s <= barrier_price:                     # 判断资产价格是否下跌触及障碍价格
                    active = True                          # 触及障碍的条件下标记合约生效
            else:                                          # 对应上涨敲入或敲出的情况
                if s >= barrier_price:                     # 判断资产价格是否上涨触及障碍价格
                    active = True                  
        if active == True:                                 # 判断生效标记，active==True即knock-in发生
            in_sum += vanilla                              # 对所有knock-in的期权价值求和(knock-out此时为0)
        else:
            out_sum += vanilla                             # 对所有knock-out的期权价值求和(knock-in此时为0)
    
    return in_sum/N_simulations, out_sum/N_simulations     # 对所有价格路径下得到的期权价值取均值即附加障碍条件以后的期权价值

in_, out_ = get_barrier_prices(s0, strike, r, q, sigma, T, 'call', barrie_price, 'up')
print(f'barrie_in price: {in_} \nbarrie_out price: {out_}')